## Installing dependencies

In [ ]:
pip install selenium webdriver-manager

In [ ]:
pip install undetected-chromedriver


## Face swapping

In [1]:
import os
import random
import time
import shutil
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from selenium.common.exceptions import (
    NoAlertPresentException, UnexpectedAlertPresentException, TimeoutException
)

# --- CONFIG ---
base_dir = r"C:\Users\adeto\Downloads\test2-20250601T005536Z-1-001\test2"
source_folder = os.path.join(base_dir, "source")
target_folder = os.path.join(base_dir, "target")
download_folder = os.path.join(base_dir, "downloads")
output_folder = os.path.join(base_dir, "deepfakes")
target_total_deepfakes = 792  # Set your desired number

os.makedirs(output_folder, exist_ok=True)
os.makedirs(download_folder, exist_ok=True)

source_images = sorted([f for f in os.listdir(source_folder) if f.endswith(".jpg")])
target_images = sorted([f for f in os.listdir(target_folder) if f.endswith(".jpg")])
existing_deepfakes = set(os.listdir(output_folder))

def dismiss_alert(driver):
    try:
        alert = driver.switch_to.alert
        print(f"⚠️ Alert: {alert.text}")
        alert.accept()
        time.sleep(1)
    except NoAlertPresentException:
        pass

def wait_for_new_download(before_files, timeout=60):
    start_time = time.time()
    while time.time() - start_time < timeout:
        current_files = set(os.listdir(download_folder))
        new_files = current_files - before_files
        if new_files:
            latest = max([os.path.join(download_folder, f) for f in new_files], key=os.path.getctime)
            if os.path.isfile(latest):
                return latest
        time.sleep(1)
    return None

def process_pair(driver, wait, src_path, tgt_path, name1, name2):
    driver.execute_script("""
        document.querySelectorAll('input[type="file"]').forEach(el => el.style.display = 'block');
    """)
    file_inputs = driver.find_elements(By.CSS_SELECTOR, "input[type='file']")
    if len(file_inputs) < 2:
        raise Exception("❌ File inputs not found.")
    
    file_inputs[0].send_keys(src_path)
    file_inputs[1].send_keys(tgt_path)
    print(f"🕒 Uploading {name1} + {name2}...")

    swap_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Start face swapping')]")))
    swap_btn.click()

    try:
        download_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "bi-download")))
    except TimeoutException:
        raise Exception("❌ Download icon not clickable.")

    parent = download_btn.find_element(By.XPATH, "./ancestor::*[self::button or self::a or self::div][1]")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", parent)
    time.sleep(1)

    before_downloads = set(os.listdir(download_folder))
    parent.click()
    print("✅ Clicked download.")

    downloaded_file = wait_for_new_download(before_downloads)
    if not downloaded_file:
        raise Exception("❌ Download failed.")

    final_name = f"swap_{name1}_{name2}.jpg"
    final_path = os.path.join(output_folder, final_name)
    shutil.move(downloaded_file, final_path)
    print(f"✅ Saved: {final_name}")
    return final_name

# --- Setup Browser ---
options = uc.ChromeOptions()
prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 40)
driver.get("https://aifaceswap.io")
time.sleep(5)

# --- Generate Until Limit is Reached ---
while len(os.listdir(output_folder)) < target_total_deepfakes:
    src_img = random.choice(source_images)
    tgt_img = random.choice(target_images)

    name1 = os.path.splitext(src_img)[0]
    name2 = os.path.splitext(tgt_img)[0]
    out_name = f"swap_{name1}_{name2}.jpg"

    if out_name in existing_deepfakes:
        print(f"⚠️ Skipping duplicate: {out_name}")
        continue

    print(f"\n🔁 Creating: {out_name}")
    try:
        src_path = os.path.join(source_folder, src_img)
        tgt_path = os.path.join(target_folder, tgt_img)
        filename = process_pair(driver, wait, src_path, tgt_path, name1, name2)
        existing_deepfakes.add(filename)
    except Exception as e:
        dismiss_alert(driver)
        print(f"❌ Error: {e}")
        try:
            driver.refresh()
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file']")))
        except:
            pass
        continue

    time.sleep(random.uniform(3, 5))
    try:
        driver.refresh()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file']")))
    except:
        continue

driver.quit()
print(f"\n🎯 Done! Total generated: {len(existing_deepfakes)}")


C:\Users\adeto\anaconda3\anaconda\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "



🔁 Creating: swap_image_00121_image_00197.jpg
🕒 Uploading image_00121 + image_00197...
✅ Clicked download.
✅ Saved: swap_image_00121_image_00197.jpg

🔁 Creating: swap_image_00283_image_00193.jpg
🕒 Uploading image_00283 + image_00193...
❌ Error: ❌ Download icon not clickable.

🔁 Creating: swap_image_00378_image_00202.jpg
🕒 Uploading image_00378 + image_00202...
✅ Clicked download.
✅ Saved: swap_image_00378_image_00202.jpg

🔁 Creating: swap_image_00188_image_00123.jpg
🕒 Uploading image_00188 + image_00123...
✅ Clicked download.
✅ Saved: swap_image_00188_image_00123.jpg

🔁 Creating: swap_image_00262_image_00217.jpg
🕒 Uploading image_00262 + image_00217...
✅ Clicked download.
✅ Saved: swap_image_00262_image_00217.jpg

🔁 Creating: swap_image_00259_image_00054.jpg
🕒 Uploading image_00259 + image_00054...
✅ Clicked download.
✅ Saved: swap_image_00259_image_00054.jpg

🔁 Creating: swap_image_00288_image_00069.jpg
🕒 Uploading image_00288 + image_00069...
✅ Clicked download.
✅ Saved: swap_image_0

✅ Clicked download.
✅ Saved: swap_image_00273_image_00254.jpg

🔁 Creating: swap_image_00131_image_00314.jpg
🕒 Uploading image_00131 + image_00314...
✅ Clicked download.
✅ Saved: swap_image_00131_image_00314.jpg

🔁 Creating: swap_image_00357_image_00071.jpg
🕒 Uploading image_00357 + image_00071...
✅ Clicked download.
✅ Saved: swap_image_00357_image_00071.jpg

🔁 Creating: swap_image_00114_image_00059.jpg
🕒 Uploading image_00114 + image_00059...
✅ Clicked download.
✅ Saved: swap_image_00114_image_00059.jpg

🔁 Creating: swap_image_00131_image_00285.jpg
🕒 Uploading image_00131 + image_00285...
✅ Clicked download.
✅ Saved: swap_image_00131_image_00285.jpg

🔁 Creating: swap_image_00335_image_00108.jpg
❌ Error: ❌ File inputs not found.

🔁 Creating: swap_image_00026_image_00267.jpg
🕒 Uploading image_00026 + image_00267...
✅ Clicked download.
✅ Saved: swap_image_00026_image_00267.jpg

🔁 Creating: swap_image_00236_image_00281.jpg
🕒 Uploading image_00236 + image_00281...
✅ Clicked download.
✅ Save


🔁 Creating: swap_image_00262_image_00373.jpg
🕒 Uploading image_00262 + image_00373...
✅ Clicked download.
✅ Saved: swap_image_00262_image_00373.jpg

🔁 Creating: swap_image_00120_image_00283.jpg
🕒 Uploading image_00120 + image_00283...
✅ Clicked download.
✅ Saved: swap_image_00120_image_00283.jpg

🔁 Creating: swap_image_00341_image_00172.jpg
🕒 Uploading image_00341 + image_00172...
✅ Clicked download.
✅ Saved: swap_image_00341_image_00172.jpg

🔁 Creating: swap_image_00272_image_00079.jpg
🕒 Uploading image_00272 + image_00079...
✅ Clicked download.
✅ Saved: swap_image_00272_image_00079.jpg

🔁 Creating: swap_image_00377_image_00363.jpg
🕒 Uploading image_00377 + image_00363...
✅ Clicked download.
✅ Saved: swap_image_00377_image_00363.jpg

🔁 Creating: swap_image_00065_image_00026.jpg
🕒 Uploading image_00065 + image_00026...
✅ Clicked download.
✅ Saved: swap_image_00065_image_00026.jpg

🔁 Creating: swap_image_00393_image_00310.jpg
🕒 Uploading image_00393 + image_00310...
✅ Clicked download.

✅ Saved: swap_image_00221_image_00317.jpg

🔁 Creating: swap_image_00041_image_00031.jpg
🕒 Uploading image_00041 + image_00031...
✅ Clicked download.
✅ Saved: swap_image_00041_image_00031.jpg

🔁 Creating: swap_image_00100_image_00321.jpg
🕒 Uploading image_00100 + image_00321...
✅ Clicked download.
✅ Saved: swap_image_00100_image_00321.jpg

🔁 Creating: swap_image_00051_image_00372.jpg
🕒 Uploading image_00051 + image_00372...
✅ Clicked download.
✅ Saved: swap_image_00051_image_00372.jpg

🔁 Creating: swap_image_00045_image_00028.jpg
🕒 Uploading image_00045 + image_00028...
✅ Clicked download.
✅ Saved: swap_image_00045_image_00028.jpg

🔁 Creating: swap_image_00249_image_00061.jpg
🕒 Uploading image_00249 + image_00061...
✅ Clicked download.
✅ Saved: swap_image_00249_image_00061.jpg

🔁 Creating: swap_image_00040_image_00368.jpg
🕒 Uploading image_00040 + image_00368...
✅ Clicked download.
✅ Saved: swap_image_00040_image_00368.jpg

🔁 Creating: swap_image_00160_image_00026.jpg
🕒 Uploading image_

...

✅ Saved: swap_image_00369_image_00048.jpg

🔁 Creating: swap_image_00382_image_00093.jpg
🕒 Uploading image_00382 + image_00093...
✅ Clicked download.
✅ Saved: swap_image_00382_image_00093.jpg

🔁 Creating: swap_image_00046_image_00036.jpg
🕒 Uploading image_00046 + image_00036...
✅ Clicked download.
✅ Saved: swap_image_00046_image_00036.jpg

🔁 Creating: swap_image_00237_image_00095.jpg
🕒 Uploading image_00237 + image_00095...
✅ Clicked download.
✅ Saved: swap_image_00237_image_00095.jpg

🔁 Creating: swap_image_00367_image_00144.jpg
🕒 Uploading image_00367 + image_00144...
✅ Clicked download.
✅ Saved: swap_image_00367_image_00144.jpg

🔁 Creating: swap_image_00372_image_00174.jpg
🕒 Uploading image_00372 + image_00174...
✅ Clicked download.
✅ Saved: swap_image_00372_image_00174.jpg

🔁 Creating: swap_image_00015_image_00365.jpg
🕒 Uploading image_00015 + image_00365...
✅ Clicked download.
✅ Saved: swap_image_00015_image_00365.jpg

🔁 Creating: swap_image_00204_image_00106.jpg
🕒 Uploading image_

✅ Clicked download.
✅ Saved: swap_image_00368_image_00197.jpg

🔁 Creating: swap_image_00150_image_00097.jpg
🕒 Uploading image_00150 + image_00097...
✅ Clicked download.
✅ Saved: swap_image_00150_image_00097.jpg

🔁 Creating: swap_image_00225_image_00131.jpg
🕒 Uploading image_00225 + image_00131...
✅ Clicked download.
✅ Saved: swap_image_00225_image_00131.jpg

🔁 Creating: swap_image_00348_image_00352.jpg
🕒 Uploading image_00348 + image_00352...
✅ Clicked download.
✅ Saved: swap_image_00348_image_00352.jpg

🔁 Creating: swap_image_00028_image_00064.jpg
🕒 Uploading image_00028 + image_00064...
✅ Clicked download.
✅ Saved: swap_image_00028_image_00064.jpg

🔁 Creating: swap_image_00077_image_00197.jpg
🕒 Uploading image_00077 + image_00197...
✅ Clicked download.
✅ Saved: swap_image_00077_image_00197.jpg

🔁 Creating: swap_image_00156_image_00144.jpg
🕒 Uploading image_00156 + image_00144...
✅ Clicked download.
✅ Saved: swap_image_00156_image_00144.jpg

🔁 Creating: swap_image_00317_image_00241.jp

✅ Clicked download.
✅ Saved: swap_image_00053_image_00318.jpg

🔁 Creating: swap_image_00306_image_00284.jpg
❌ Error: ❌ File inputs not found.

🔁 Creating: swap_image_00214_image_00299.jpg
🕒 Uploading image_00214 + image_00299...
✅ Clicked download.
✅ Saved: swap_image_00214_image_00299.jpg

🔁 Creating: swap_image_00229_image_00193.jpg
🕒 Uploading image_00229 + image_00193...
✅ Clicked download.
✅ Saved: swap_image_00229_image_00193.jpg

🔁 Creating: swap_image_00186_image_00136.jpg
🕒 Uploading image_00186 + image_00136...
✅ Clicked download.
✅ Saved: swap_image_00186_image_00136.jpg

🔁 Creating: swap_image_00193_image_00045.jpg
🕒 Uploading image_00193 + image_00045...
✅ Clicked download.
✅ Saved: swap_image_00193_image_00045.jpg

🔁 Creating: swap_image_00089_image_00156.jpg
🕒 Uploading image_00089 + image_00156...
✅ Clicked download.
✅ Saved: swap_image_00089_image_00156.jpg

🔁 Creating: swap_image_00348_image_00238.jpg
🕒 Uploading image_00348 + image_00238...
✅ Clicked download.
✅ Save

🕒 Uploading image_00379 + image_00217...
✅ Clicked download.
✅ Saved: swap_image_00379_image_00217.jpg

🔁 Creating: swap_image_00254_image_00307.jpg
🕒 Uploading image_00254 + image_00307...
✅ Clicked download.
✅ Saved: swap_image_00254_image_00307.jpg

🔁 Creating: swap_image_00226_image_00119.jpg
🕒 Uploading image_00226 + image_00119...
✅ Clicked download.
✅ Saved: swap_image_00226_image_00119.jpg

🔁 Creating: swap_image_00350_image_00163.jpg
🕒 Uploading image_00350 + image_00163...
✅ Clicked download.
✅ Saved: swap_image_00350_image_00163.jpg

🔁 Creating: swap_image_00385_image_00189.jpg
🕒 Uploading image_00385 + image_00189...
✅ Clicked download.
✅ Saved: swap_image_00385_image_00189.jpg

🔁 Creating: swap_image_00252_image_00109.jpg
🕒 Uploading image_00252 + image_00109...
✅ Clicked download.
✅ Saved: swap_image_00252_image_00109.jpg

🔁 Creating: swap_image_00005_image_00334.jpg
🕒 Uploading image_00005 + image_00334...
✅ Clicked download.
✅ Saved: swap_image_00005_image_00334.jpg

🔁 

✅ Clicked download.
✅ Saved: swap_image_00184_image_00257.jpg

🔁 Creating: swap_image_00161_image_00015.jpg
🕒 Uploading image_00161 + image_00015...
✅ Clicked download.
✅ Saved: swap_image_00161_image_00015.jpg

🔁 Creating: swap_image_00099_image_00156.jpg
🕒 Uploading image_00099 + image_00156...
✅ Clicked download.
✅ Saved: swap_image_00099_image_00156.jpg

🔁 Creating: swap_image_00395_image_00195.jpg
🕒 Uploading image_00395 + image_00195...
✅ Clicked download.
✅ Saved: swap_image_00395_image_00195.jpg

🔁 Creating: swap_image_00256_image_00143.jpg
🕒 Uploading image_00256 + image_00143...
✅ Clicked download.
✅ Saved: swap_image_00256_image_00143.jpg

🔁 Creating: swap_image_00186_image_00352.jpg
🕒 Uploading image_00186 + image_00352...
✅ Clicked download.
✅ Saved: swap_image_00186_image_00352.jpg

🔁 Creating: swap_image_00238_image_00259.jpg
🕒 Uploading image_00238 + image_00259...
✅ Clicked download.
✅ Saved: swap_image_00238_image_00259.jpg

🎯 Done! Total generated: 792


## Creating real folders by collapsing the source and target folders

In [ ]:
import os
import shutil

# Define paths
base_dir = r"C:\Users\adeto\Downloads\train2-20250601T005515Z-1-001\train2"
source_folder = os.path.join(base_dir, "source")
target_folder = os.path.join(base_dir, "target")
real_folder = os.path.join(base_dir, "real")  

os.makedirs(real_folder, exist_ok=True)

# Copy all source images first
src_images = sorted([f for f in os.listdir(source_folder) if f.lower().endswith(".jpg")])
for img in src_images:
    src_path = os.path.join(source_folder, img)
    dst_path = os.path.join(real_folder, img)
    shutil.copy(src_path, dst_path)

print(f"✅ Copied {len(src_images)} images from source.")

# Determine the last index used in source filenames
import re

def extract_number(filename):
    nums = re.findall(r'\d+', filename)
    return int(nums[-1]) if nums else -1

max_index = max([extract_number(f) for f in src_images if extract_number(f) != -1], default=0)
print(f"ℹ️ Last image number in source: {max_index}")

# Copy target images, renaming them to avoid collisions
tgt_images = sorted([f for f in os.listdir(target_folder) if f.lower().endswith(".jpg")])
new_index = max_index + 1

for img in tgt_images:
    tgt_path = os.path.join(target_folder, img)
    new_name = f"image{new_index:04d}.jpg"
    dst_path = os.path.join(real_folder, new_name)
    shutil.copy(tgt_path, dst_path)
    new_index += 1

print(f"✅ Copied and renamed {len(tgt_images)} images from target.")
print(f"🎉 Total real images in '{real_folder}': {len(os.listdir(real_folder))}")


In [ ]:
import os
import shutil

# Define paths
base_dir = r"C:\Users\adeto\Downloads\val2-20250601T005515Z-1-001\val2"
source_folder = os.path.join(base_dir, "source")
target_folder = os.path.join(base_dir, "target")
real_folder = os.path.join(base_dir, "real")  

os.makedirs(real_folder, exist_ok=True)

# Copy all source images first
src_images = sorted([f for f in os.listdir(source_folder) if f.lower().endswith(".jpg")])
for img in src_images:
    src_path = os.path.join(source_folder, img)
    dst_path = os.path.join(real_folder, img)
    shutil.copy(src_path, dst_path)

print(f"✅ Copied {len(src_images)} images from source.")

# Determine the last index used in source filenames
import re

def extract_number(filename):
    nums = re.findall(r'\d+', filename)
    return int(nums[-1]) if nums else -1

max_index = max([extract_number(f) for f in src_images if extract_number(f) != -1], default=0)
print(f"ℹ️ Last image number in source: {max_index}")

# Copy target images, renaming them to avoid collisions
tgt_images = sorted([f for f in os.listdir(target_folder) if f.lower().endswith(".jpg")])
new_index = max_index + 1

for img in tgt_images:
    tgt_path = os.path.join(target_folder, img)
    new_name = f"image{new_index:04d}.jpg"
    dst_path = os.path.join(real_folder, new_name)
    shutil.copy(tgt_path, dst_path)
    new_index += 1

print(f"✅ Copied and renamed {len(tgt_images)} images from target.")
print(f"🎉 Total real images in '{real_folder}': {len(os.listdir(real_folder))}")


In [1]:
import os
import shutil

# Define paths
base_dir = r"C:\Users\adeto\Downloads\test2-20250601T005536Z-1-001\test2"
source_folder = os.path.join(base_dir, "source")
target_folder = os.path.join(base_dir, "target")
real_folder = os.path.join(base_dir, "real")  

os.makedirs(real_folder, exist_ok=True)

# Copy all source images first
src_images = sorted([f for f in os.listdir(source_folder) if f.lower().endswith(".jpg")])
for img in src_images:
    src_path = os.path.join(source_folder, img)
    dst_path = os.path.join(real_folder, img)
    shutil.copy(src_path, dst_path)

print(f"✅ Copied {len(src_images)} images from source.")

# Determine the last index used in source filenames
import re

def extract_number(filename):
    nums = re.findall(r'\d+', filename)
    return int(nums[-1]) if nums else -1

max_index = max([extract_number(f) for f in src_images if extract_number(f) != -1], default=0)
print(f"ℹ️ Last image number in source: {max_index}")

# Copy target images, renaming them to avoid collisions
tgt_images = sorted([f for f in os.listdir(target_folder) if f.lower().endswith(".jpg")])
new_index = max_index + 1

for img in tgt_images:
    tgt_path = os.path.join(target_folder, img)
    new_name = f"image{new_index:04d}.jpg"
    dst_path = os.path.join(real_folder, new_name)
    shutil.copy(tgt_path, dst_path)
    new_index += 1

print(f"✅ Copied and renamed {len(tgt_images)} images from target.")
print(f"🎉 Total real images in '{real_folder}': {len(os.listdir(real_folder))}")


✅ Copied 408 images from source.
ℹ️ Last image number in source: 408
✅ Copied and renamed 384 images from target.
🎉 Total real images in 'C:\Users\adeto\Downloads\test2-20250601T005536Z-1-001\test2\real': 792
